# Create BernCrop File

This notebook is used to create the HDF5 File for the BernCrop Dataset.

## Define Functions and Imports

In [ ]:
from sentinelhub import SHConfig
import h5py
import torch
import datetime
import pandas as pd

import matplotlib.pyplot as plt

# set default plt figsize to (12,6)
plt.rcParams["figure.figsize"] = (12,6)
pd.set_option('display.colheader_justify', 'center')

from sentinelhub import (
    BBox,
    DataCollection,
    MimeType,
    MosaickingOrder,
    SentinelHubRequest,
    bbox_to_dimensions,
    SentinelHubDownloadClient
)

download_log =''

# Define the HDF5 file path and dataset parameters
file_name_bern = r'..\raw_data\BernCrop\BernCrop.hdf5'
file_name_zueri = r'..\raw_data\ZueriCrop\ZueriCrop.hdf5'

In [ ]:
def create_config(instance_id='8b5b5797-c269-42a4-b75a-dc1ed7e469a8',
                  sh_client_id='f00771eb-624d-4106-973a-98963b395356',
                  sh_client_secret='vnUy3ozu_s[z}W1O/>l6%U6GsfLfs]tLeS@jBs%!',
                  sh_base_url='https://services.sentinel-hub.com',
                  ):
    config = SHConfig(
        instance_id=instance_id,
        sh_client_id=sh_client_id,
        sh_client_secret=sh_client_secret,
        sh_base_url=sh_base_url,
    )
    if not config.sh_client_id or not config.sh_client_secret:
        print("Warning! To use Process API, please provide the credentials (OAuth client ID and client secret).")
    return config
def create_bbox(coordinates:tuple, resolution:int=10, crs:int=32632):
    resolution = resolution
    bbox = BBox(bbox=coordinates, crs=crs)
    bbox_size = bbox_to_dimensions(bbox, resolution=resolution)
    print(f"Image shape at {resolution} m resolution: {bbox_size} pixels")
    return bbox, bbox_size

def create_time_slots(n_chunks :int = 365,
                      start_date :datetime.datetime = datetime.datetime(2022, 1, 1,1,1,1,1),
                      end_date :datetime.datetime = datetime.datetime(2022, 12, 31,1,1,1,1),verbose:bool =False):
    tdelta = (end_date - start_date) / n_chunks
    edges = [(start_date + i * tdelta).date().isoformat() for i in range(n_chunks)]
    slots = [(edges[i], edges[i + 1]) for i in range(len(edges) - 1)]

    if verbose:
        print("time windows:\n")
        for slot in slots:
            print(slot)
    return slots

def get_rgbnir_color_request(time_interval, coordinates:tuple):

    evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04", "B08"],
                units: "DN"
            }],
        output: { // this defines the output image type
        bands: 4, // the output of this evalscript will have RGB colors
        sampleType: "UINT16" // raster format will be UINT16
        }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B04,sample.B03,sample.B02 , sample.B08];
    }
"""
    bbox, bbox_size = create_bbox(coordinates)
    return SentinelHubRequest(
        evalscript=evalscript_true_color,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L2A,
                time_interval=time_interval,
                mosaicking_order=MosaickingOrder.LEAST_CC,
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
        bbox=bbox,
        size=bbox_size,
        config=create_config(),
    )


def get_coords():
    coords = pd.read_csv(r'..\raw_data\BernCrop\bern_bboxes_sentinel.csv',index_col=None)
    coords = coords[coords['area'] >0]
    coords= coords.iloc[:,1:5]
    return coords


def plot_seasons(dataset):
    import numpy as np
    import matplotlib.pyplot as plt

    # Sample data (replace this with your actual data)
    # data= np.random.rand(287, 24, 24, 3, 10)  # 4 time steps, 3x3 pixels, 5 channels, 10 samples
    data = torch.tensor(dataset).permute(1,2,3,4,0)
    # Assuming you have information about which time steps belong to which seasons
    # In this example, let's say time steps are equally divided into seasons
    # Adjust this based on your actual data and seasons definition
    seasons = ['Spring', 'Summer', 'Autumn', 'Winter']
    time_steps_per_season = len(data) // len(seasons)

    # Initialize histograms for each season
    histograms = {season: np.zeros((24, 24, 9, 10), dtype=int) for season in seasons}

    # Group data into histograms by season
    for season, start_idx in zip(seasons, range(0, len(data), time_steps_per_season)):
        end_idx = start_idx + time_steps_per_season
        data_slice = data[start_idx:end_idx]
        season_histogram = np.histogram(data_slice, bins=256, range=(0, 256))
        histograms[season] = season_histogram

    # Visualize histograms using bar charts
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    fig.suptitle('Histograms for Different Seasons')

    for i, season in enumerate(seasons):
        ax = axes[i // 2, i % 2]
        ax.set_title(season)
        ax.bar(range(256), histograms[season][0], width=0.8, align='center', color='skyblue')
        ax.set_xlabel('Channel Values')
        ax.set_ylabel('Frequency')

    plt.tight_layout()
    plt.show()


## Get Satellite Data from Sentinel

In [ ]:
# create sentinel config
config=create_config()

### Get Bounding-Boxes 

In [ ]:
# TODO: Select coords
coords = get_coords().iloc[0:1]
coords

In [ ]:
coords['x1'] - coords['x2']
coords['x2'] = coords['x1'] + 240

In [ ]:
coords['y1'] - coords['y2']
coords['y2'] = coords['y1'] + 240

In [ ]:
coords['y1'] - coords['y2']

In [ ]:
coords['x1'] - coords['x2']

## Download and put satellite data into hdf5 file

We will download data from Sentinel-hub in the size 240x240 pixel and split them afterwards into 100 24x24 pixel fields.
These fields are then stored in the hdf5 file.

In [ ]:
%%capture download_log --no-stderr
print(f"Start Download at: {datetime.datetime.today()}")

temp_results_tensor = []
twoforty_tensor = False
for i,coordinates in coords.iterrows():
    print(f'started downloading bounding_box: {coordinates} with index:{i}')
    
    # assert bbox_to_dimensions(BBox(bbox=tuple(coordinates), crs=32632), resolution=10) == (240,240)
    
    list_of_requests = [get_rgbnir_color_request(time_interval, tuple(coordinates)) for time_interval in create_time_slots(142)]
    list_of_requests = [request.download_list[0] for request in list_of_requests]
    
    # download data with multiple threads
    download_data_timesteps = SentinelHubDownloadClient(config=config).download(list_of_requests, max_threads=20)
    non_zero_downloads = torch.tensor([v for i,v in enumerate(download_data_timesteps) if download_data_timesteps[i].sum() > 0]).unsqueeze(0)
    temp_results_tensor.append(non_zero_downloads)

result_tensor = torch.cat(temp_results_tensor, dim=0)
print(f"Stopped Download at: {datetime.datetime.today()}")

print(f"Start Reshape Data at: {datetime.datetime.today()}")

# Reshape the original tensor into the target shape
# result_shape = (result_tensor.shape[0]*100, result_tensor.shape[1], 24, 24, result_tensor.shape[4])
# result_tensor = result_tensor.reshape(result_shape)
print(f"Stopped Reshape at: {datetime.datetime.today()}")


print(f"Start Saving Data at: {datetime.datetime.today()}")


dataset_data_name = "data"
data_shape = result_tensor.shape


with h5py.File(file_name_bern, 'a') as hf:
    # Check if the dataset already exists
    if dataset_data_name in hf:
        dataset = hf[dataset_data_name]
    else:
        dtype = "float32"  # Use the appropriate data type for your data
        dataset = hf.create_dataset(dataset_data_name, shape=(0,) + data_shape[1:], dtype=dtype, maxshape=(None,) + data_shape[1:])
        
    current_size = dataset.shape[0]
    new_size = current_size + result_tensor.shape[0]
    # Resize the dataset to accommodate the new batch
    dataset.resize(new_size, axis=0)
    # Append the new batch to the dataset
    dataset[current_size:new_size, :] = result_tensor

print(f"Stopped Saving Data at: {datetime.datetime.today()}")

In [ ]:
result_tensor.shape

## Label Data to HDF5
Now we add the label data to the 'gt' hdf5 file dataset.

In [ ]:
dataset_label_name = "gt"
#TODO: define label_tensor
label_shape = result_tensor[:,-1,:,:,-1].shape
label_tensor = result_tensor[:,-1,:,:,-1]

with h5py.File(file_name_bern, 'a') as hf:    
    # Check if the dataset already exists
    if dataset_label_name in hf:
        dataset = hf[dataset_label_name]
    else:
        dtype = "float32"  # Use the appropriate data type for your data
        dataset = hf.create_dataset(dataset_label_name, shape=(0,) + label_shape[1:], dtype=dtype, maxshape=(None,) + label_shape[1:])
        
    current_size = dataset.shape[0]
    new_size = current_size + label_tensor.shape[0]
    # Resize the dataset to accommodate the new batch
    dataset.resize(new_size, axis=0)
    # Append the new batch to the dataset
    dataset[current_size:new_size, :] = label_tensor

print(f"Stopped Saving Data at: {datetime.datetime.today()}")

In [ ]:
# Open the HDF5 file in read mode
with h5py.File(file_name_bern, "r") as file:
    # Check if the "data" dataset exists in the file
    if "data" in file:
        # Access the dataset and read its contents into a NumPy array
        dataset_b = file["data"][:]
    else:
        print("Dataset 'data' not found in the HDF5 file.")


In [ ]:
dataset_b[:,:,:,:,0:1].shape

In [ ]:
plot_seasons(dataset_b[:,:,:,:,:])

In [ ]:
plot_seasons(dataset_b[:,:,:,:,0:1])

In [ ]:
plot_seasons(dataset_b[:,:,:,:,1:2])

In [ ]:
plot_seasons(dataset_b[:,:,:,:,2:3])

In [ ]:
plot_seasons(dataset_b[:,:,:,:,3:4])

In [ ]:
# Open the HDF5 file in read mode
with h5py.File(file_name_zueri, "r") as file:
    # Check if the "data" dataset exists in the file
    if "data" in file:
        # Access the dataset and read its contents into a NumPy array
        dataset_z = file["data"][0:1000]
    else:
        print("Dataset 'data' not found in the HDF5 file.")


In [ ]:
dataset_z.shape
plot_seasons(dataset_z)

In [ ]:
# Open the HDF5 file in read mode
with h5py.File(file_name_bern, "r") as file:
    # Check if the "data" dataset exists in the file
    if "data" in file:
        # Access the dataset and read its contents into a NumPy array
        dataset = file["gt"][:]
    else:
        print("Dataset 'data' not found in the HDF5 file.")


In [ ]:
dataset.shape